# Init

In [1]:
# Import utils
import numpy as np
import pandas as pd
import math
import time
import json
import pyreadr
import pickle
from joblib import dump, load
import os
import copy
import datetime


# Import Gurobi
import gurobipy as gp
from gurobipy import GRB

# Optimization Module
from DataDrivenPatientScheduling import PatientScheduling

# Preprocessing

# Conditional scenario sampling based on weights

For each patient $j=1,...,M$ for the current day $t$, we have $N_t$ historical samples. Given patient $j$ and the associated feature vector $\boldsymbol{x}_{j,t}$, we have $N_t$ weights for these samples. With this, we have an approximation of the empirical probability distribution of surgery duration. We use the approximated distribution to draw $K$ scenarios for each patient $j=1,...,M$.

In [2]:
M = 10
N = 250
K = 10**5

# Dummy samples (to be replaced)
samples = np.random.rand(M, N) * 48

# Dummy weights (to be replaced)
weights = np.random.rand(M, N)
weights = (1 / np.sum(weights, axis=1)).reshape(-1,1) * weights

In [3]:
# Draw
scenarios = []
for k in range(K):    
    scenario = []
    for j in range(M):
        scenario += [np.random.choice(samples[j,].flatten(), p=weights[j,].flatten())]
        
    scenarios += [scenario]

In [4]:
scenarios = np.array(scenarios)
scenarios.shape

(100000, 10)

In [5]:
# Initialize model
ddps = PatientScheduling(c_waitingtime=1, c_overtime=1, T=600)

In [6]:
# Set up model with scenarios
ddps.create(z=scenarios)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-04


In [7]:
# Solve
result = ddps.optimize(LogToConsole=1, Threads=32)

Set parameter Threads to value 32
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD EPYC 7713P 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads

Optimize a model with 1000001 rows, 1100010 columns and 3000010 nonzeros
Model fingerprint: 0xe2ac39c4
Variable types: 0 continuous, 1100010 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e-05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-03, 6e+02]
Found heuristic solution: objective 120.2667800
Presolve removed 100000 rows and 200001 columns
Presolve time: 4.80s
Presolved: 900001 rows, 900009 columns, 2600009 nonzeros
Variable types: 0 continuous, 900009 integer (1322 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 1.92s

Barrier statistics:
 Dense cols : 9
 AA' N

In [9]:
# Apply schedule
schedule, times, status, solutions, gap = result

In [10]:
schedule

array([  0., 216., 264., 312., 360., 408., 456., 504., 552., 600.])